## 01/03/19

- Developing new features using visualisation, specifically maximum energy associated ones.

In [1]:
import sys
sys.path.insert(0, '/Users/matthewashman/github/MasterProject2018')

# Import necessary modules. Set settings. Import data.
import math
import numpy as np
import pandas as pd
import random
import pywt
import matplotlib.pyplot as plt
from statsmodels.robust import mad
from tsfresh.feature_extraction import feature_calculators
from FeatureExtraction.feature_tools import detect_peaks
from IPython.display import display, clear_output, HTML

import pdb

plt.style.use('default')

X = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/Data/X_all_labels.pkl')

/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Remove bad files with bad labels
X = X[~(X['CS1-2 Label']=='-1') & ~(X['CS3-4 Label']=='-1') & ~(X['CS5-6 Label']=='-1')]

In [ ]:
%matplotlib qt
patient_types = X['Type'].unique()
M=14
v = np.ones(M)

for patient_type in patient_types:
    patient_type = 'avrt'
    patients = X[X['Type']==patient_type]['Patient'].unique()
    patients = patients[1:]
    for patient in patients:
        patient_X = X[(X['Type']==patient_type) & (X['Patient']==patient) & (X['S1/S2']=='S2')]
        
        patient_cis = patient_X['Coupling Interval'].unique()

        fig, axes = plt.subplots(nrows=len(patient_cis), ncols=3, sharex=True, figsize=(16,9))
        
        for i,ci in enumerate(patient_cis):
            cs12 = patient_X[patient_X['Coupling Interval']==ci]['CS1-2'].values[0]
            cs34 = patient_X[patient_X['Coupling Interval']==ci]['CS3-4'].values[0]
            cs56 = patient_X[patient_X['Coupling Interval']==ci]['CS5-6'].values[0]
            
            cs12_ = np.convolve(abs(cs12), v, mode='valid')
            cs34_ = np.convolve(abs(cs34), v, mode='valid')
            cs56_ = np.convolve(abs(cs56), v, mode='valid')
            
            width_thresh=0.4
            
            if any(cs12_[np.argmax(cs12_):] < width_thresh*np.max(cs12_)):
                cs12_end_idx = np.argmax(cs12_) + np.argmax(cs12_[np.argmax(cs12_):] < width_thresh*np.max(cs12_))
            else:
                cs12_end_idx = len(cs12_)-1
            if any(cs12_[np.argmax(cs12_)::-1] < width_thresh*np.max(cs12_)):  
                cs12_start_idx = np.argmax(cs12_) - np.argmax(cs12_[np.argmax(cs12_)::-1] < width_thresh*np.max(cs12_))
            else:
                cs12_start_idx = 0
            if any(cs34_[np.argmax(cs34_):] < width_thresh*np.max(cs34_)):
                cs34_end_idx = np.argmax(cs34_) + np.argmax(cs34_[np.argmax(cs34_):] < width_thresh*np.max(cs34_))
            else:
                cs34_end_idx = len(cs34_)-1
            if any(cs34_[np.argmax(cs34_)::-1] < width_thresh*np.max(cs34_)):  
                cs34_start_idx = np.argmax(cs34_) - np.argmax(cs34_[np.argmax(cs34_)::-1] < width_thresh*np.max(cs34_))
            else:
                cs34_start_idx = 0
            if any(cs56_[np.argmax(cs56_):] < width_thresh*np.max(cs56_)):
                cs56_end_idx = np.argmax(cs56_) + np.argmax(cs56_[np.argmax(cs56_):] < width_thresh*np.max(cs56_))
            else:
                cs56_end_idx = len(cs56_)-1
            if any(cs56_[np.argmax(cs56_)::-1] < width_thresh*np.max(cs56_)):  
                cs56_start_idx = np.argmax(cs56_) - np.argmax(cs56_[np.argmax(cs56_)::-1] < width_thresh*np.max(cs56_))
            else:
                cs56_start_idx = 0
                        
            axes[i][0].plot(np.arange(len(cs12)), cs12/max(abs(cs12)), '--', np.arange(len(cs12_))+M/2, cs12_/max(abs(cs12_)))
            axes[i][0].axvline(x=np.argmax(cs12_)+M/2, color='k', linestyle='--')
            axes[i][0].axvline(x=cs12_start_idx + M/2, color='r', linestyle='--')
            axes[i][0].axvline(x=cs12_end_idx+M/2, color='r', linestyle='--')
            
            axes[i][1].plot(np.arange(len(cs34)), cs34/max(abs(cs34)), '--', np.arange(len(cs34_))+M/2, cs34_/max(abs(cs34_)))
            axes[i][1].axvline(x=np.argmax(cs34_)+M/2, color='k', linestyle='--')
            axes[i][1].axvline(x=cs34_start_idx + M/2, color='r', linestyle='--')
            axes[i][1].axvline(x=cs34_end_idx+M/2, color='r', linestyle='--')
            
            axes[i][2].plot(np.arange(len(cs56)), cs56/max(abs(cs56)), '--', np.arange(len(cs56_))+M/2, cs56_/max(abs(cs56_)))
            axes[i][2].axvline(x=np.argmax(cs56_)+M/2, color='k', linestyle='--')
            axes[i][2].axvline(x=cs56_start_idx + M/2, color='r', linestyle='--')
            axes[i][2].axvline(x=cs56_end_idx+M/2, color='r', linestyle='--')
            
        
        plt.suptitle('Convolved signal for: ' + patient_type + ' ' + patient)
        plt.draw()
        plt.waitforbuttonpress()
        plt.close()
    

In [ ]:
%matplotlib qt
patient_types = X['Type'].unique()

for patient_type in patient_types:
    patients = X[X['Type']==patient_type]['Patient'].unique()
    if patient_type == 'avrt':
        patients = patients[1:]
        
    for patient in patients:
        patient_X = X[(X['Type']==patient_type) & (X['Patient']==patient) & (X['S1/S2']=='S2')]
        
        patient_cis = patient_X['Coupling Interval'].unique()

        fig, axes = plt.subplots(nrows=len(patient_cis), ncols=3, sharex=True, figsize=(16,9))
        
        for i,ci in enumerate(patient_cis):
            cs12 = patient_X[patient_X['Coupling Interval']==ci]['CS1-2'].values[0]
            cs34 = patient_X[patient_X['Coupling Interval']==ci]['CS3-4'].values[0]
            cs56 = patient_X[patient_X['Coupling Interval']==ci]['CS5-6'].values[0]
            
            cs12_peaks = get_peaks(cs12, 0.1)
            cs34_peaks = get_peaks(cs34, 0.1)
            cs56_peaks = get_peaks(cs56, 0.1)
            
            axes[i][0].plot(np.arange(len(cs12)), cs12)
            axes[i][1].plot(np.arange(len(cs34)), cs34)
            axes[i][2].plot(np.arange(len(cs56)), cs56)
            for idx, amp in cs12_peaks:
                axes[i][0].scatter(idx, amp, 'kx')
            
            
            axes[i][0].axvline(x=np.argmax(cs12_)+M/2, color='k', linestyle='--')
            axes[i][0].axvline(x=cs12_start_idx + M/2, color='r', linestyle='--')
            axes[i][0].axvline(x=cs12_end_idx+M/2, color='r', linestyle='--')
            
            axes[i][1].plot(np.arange(len(cs34)), cs34/max(abs(cs34)), '--', np.arange(len(cs34_))+M/2, cs34_/max(abs(cs34_)))
            axes[i][1].axvline(x=np.argmax(cs34_)+M/2, color='k', linestyle='--')
            axes[i][1].axvline(x=cs34_start_idx + M/2, color='r', linestyle='--')
            axes[i][1].axvline(x=cs34_end_idx+M/2, color='r', linestyle='--')
            
            axes[i][2].plot(np.arange(len(cs56)), cs56/max(abs(cs56)), '--', np.arange(len(cs56_))+M/2, cs56_/max(abs(cs56_)))
            axes[i][2].axvline(x=np.argmax(cs56_)+M/2, color='k', linestyle='--')
            axes[i][2].axvline(x=cs56_start_idx + M/2, color='r', linestyle='--')
            axes[i][2].axvline(x=cs56_end_idx+M/2, color='r', linestyle='--')
            
        
        plt.suptitle('Convolved signal for: ' + patient_type + ' ' + patient)
        plt.draw()
        plt.waitforbuttonpress()
        plt.close()
    

In [3]:
def denoise(x):
    # Obtain Daubechies N=6 wavelet coefficients
    waveletCoefs = pywt.wavedec(x, 'db7', mode='per')

    # Throw away coefficients corresponding to noise
    sigma = mad(waveletCoefs[-1])
    uThresh = 1*sigma*np.sqrt(2*np.log(len(x)))
    denoised = waveletCoefs[:]
    denoised[1:] = (pywt._thresholding.hard(i, value=uThresh) for i in denoised[1:])

    # Reconstruct the original signal
    xDenoised = pywt.waverec(denoised, 'db7', mode='per')

    return xDenoised

def get_peaks(x, height_thresh, scale_amp=None, set_scale=False, plot = False):
    x = np.array(x)
    
    # Get height_thresh
    if set_scale:
        height_thresh = height_thresh*scale_amp
    else:
        height_thresh = height_thresh*max(abs(x))
    
    # Denoise x
    xdn = denoise(x)

    # Detect peaks using detect_peaks
    pos_peak_idx = detect_peaks(xdn, mph=height_thresh, threshold = 0)
    neg_peak_idx = detect_peaks((-xdn), mph=height_thresh, threshold = 0)
    peak_idx = np.concatenate([pos_peak_idx, neg_peak_idx])
    peak_idx = np.sort(peak_idx)
    # Edge indeces aren't detected
    peak_idx = peak_idx[(peak_idx != 0) & (peak_idx != (len(xdn)-1))]

    new_peak_idx = []
    peak_amp = []
    if (len(peak_idx) > 0):
        new_peak_idx.append(peak_idx[0])
        mp_thresh = 0.2*max(abs(x))
        for i in range(len(peak_idx)-1):
            idx = peak_idx[i]
            idx_next = peak_idx[i+1]
            mid_point = int((idx_next+idx)/2)
            if (max([abs(x[idx_next]-x[mid_point]), abs(x[idx]-x[mid_point])]) > mp_thresh):
                new_peak_idx.append(idx_next)

        peak_idx = np.array(new_peak_idx)
        peak_amp = x[peak_idx]

    if plot == True:
        fig, [ax1] = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(8,8))
        ax1.plot(x, 'b' , xdn, 'r--', peak_idx, peak_amp, 'kx')
        #plt.title(fileName)
        ax1.set_xlabel('Sample')
        ax1.set_ylabel('Normalised amplitude')
        ax1.legend(['Original segment', 'Denoised segment', 'Detected peaks'])

        plt.draw()
        plt.waitforbuttonpress(0) # this will wait for indefinite time
        plt.close(fig)


    return peak_idx, peak_amp